In [3]:
import numpy as np
import pandas as pd

from astropy.table import Table
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy import wcs
from astropy.io import ascii

from astroquery.sdss import SDSS

import datetime as dt
import glob
import tarfile
import os
import sys
import time
import re
import json

try: # Python 3.x
    from urllib.parse import quote as urlencode
    from urllib.request import urlretrieve
except ImportError:  # Python 2.x
    from urllib import pathname2url as urlencode
    from urllib import urlretrieve

try: # Python 3.x
    import http.client as httplib 
except ImportError:  # Python 2.x
    import httplib   

import pprint
pp = pprint.PrettyPrinter(indent=4)

# Andrews SDSS MZR Data

In [4]:
and_header = pd.read_table('/Users/Briana/Documents/Grad_School/HPS/comparison_populations/SDSS_MZR/andrews_header.ascii', delim_whitespace=True)
col_names = and_header['Label'].values
print col_names

['logM*lo' 'logM*up' 'logSFRlo' 'logSFRup' 'Ngal' 'logM*' 'logSFR'
 'OII3727' 'e_OII3727' 'NeIII3868' 'e_NeIII3868' 'SII4069' 'e_SII4069'
 'Hg' 'e_Hg' 'OIII4363' 'e_OIII4363' 'HeII4686' 'e_HeII4686' 'ArIV4740'
 'e_ArIV4740' 'OIII4959' 'e_OIII4959' 'OIII5007' 'e_OIII5007' 'NII5755'
 'e_NII5755' 'SIII6312' 'e_SIII6312' 'NII6548' 'e_NII6548' 'Ha' 'e_Ha'
 'NII6583' 'e_NII6583' 'SII6716' 'e_SII6716' 'SII6731' 'e_SII6731'
 'ArIII7135' 'e_ArIII7135' 'OII7320' 'e_OII7320' 'OII7330' 'e_OII7330']


/Users/Briana/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


In [5]:
#units of Hb = 100, logM* is the median stacked mass 
andrews = pd.read_fwf('/Users/Briana/Documents/Grad_School/HPS/comparison_populations/SDSS_MZR/andrews_flux_table.ascii', names=col_names)
andrews_ref = andrews[~np.isnan(andrews['OII3727']) & ~np.isnan(andrews['OIII5007'])]
print andrews.columns

andrews.to_csv('/Users/Briana/Documents/Grad_School/HPS/comparison_populations/SDSS_MZR/andrews_flux_table.csv')
#print andrews_ref.loc[:, ['logM*', 'Ngal', 'OII3727', 'OIII5007', 'NeIII3868']]
#print andrews['Ngal']

Index([u'logM*lo', u'logM*up', u'logSFRlo', u'logSFRup', u'Ngal', u'logM*',
       u'logSFR', u'OII3727', u'e_OII3727', u'NeIII3868', u'e_NeIII3868',
       u'SII4069', u'e_SII4069', u'Hg', u'e_Hg', u'OIII4363', u'e_OIII4363',
       u'HeII4686', u'e_HeII4686', u'ArIV4740', u'e_ArIV4740', u'OIII4959',
       u'e_OIII4959', u'OIII5007', u'e_OIII5007', u'NII5755', u'e_NII5755',
       u'SIII6312', u'e_SIII6312', u'NII6548', u'e_NII6548', u'Ha', u'e_Ha',
       u'NII6583', u'e_NII6583', u'SII6716', u'e_SII6716', u'SII6731',
       u'e_SII6731', u'ArIII7135', u'e_ArIII7135', u'OII7320', u'e_OII7320',
       u'OII7330', u'e_OII7330'],
      dtype='object')


In [6]:
andrews_red = andrews[~np.isnan(andrews['Ha'])]
#print andrews_red.loc[:, ['logM*', 'Ngal', 'Ha']]

# Full HPS Vizier Catalog

In [7]:
hps_full = pd.read_csv('/Users/Briana/Documents/Grad_School/HPS/HPS_dataframes/HPS_full_VizieR_table.dat', delim_whitespace=True)
hps_full[hps_full['HPS_ID']==366]
#hps_full.iloc[420:440]

,HPS_ID,lambda,FWHM,S_N,Flux,Spat,mag,n_mag,Prob,EW,EWint,Trans,z,Lya,X-ray,RAJ2000,DEJ2000
365,366,4020.31,310,14.7,87.6,5.8,20.55,R,1.0,20.0,39.1,[OII],0.0787,0.0,J1236331+621134,12 36 33.08,+62 11 33.3


# HPS OII Emitter catalog

In [8]:
hps = pd.read_csv('/Users/Briana/Documents/Grad_School/HPS/HPS_dataframes/HPS_cat_table.dat')

#print hps.sort_values(by=['OII_ID'])

#These are the possible AGN that were thrown out
poss_AGN = hps[(hps['OII_ID']==73.0) | (hps['OII_ID']==366.0)]
print poss_AGN
poss_AGN.to_csv('/Users/Briana/Documents/Grad_School/HPS/HPS_dataframes/HPS_AGN.csv')

hps_z = hps.loc[:,['HPS_name', 'OII_ID', 'OII_z_x', 'OIII1_ID', 'OIII1_z']]
hps_z.rename(columns={'OII_ID':'ID', 'OII_z_x':'z'}, inplace=True)
hps_z.at[4, 'ID'] = hps_z.loc[4,['OIII1_ID']]
hps_z.at[4, 'z'] = hps_z.loc[4,['OIII1_z']]
hps_z.at[5, 'ID'] = hps_z.loc[5,['OIII1_ID']]
hps_z.at[5, 'z'] = hps_z.loc[5,['OIII1_z']]
hps_z = hps_z.loc[:,['HPS_name','ID','z']]
#print hps_z

hps_z.to_csv('/Users/Briana/Documents/Grad_School/HPS/HPS_dataframes/HPS_z.dat')
#print hps[hps['HPS_name'] == 'HPS100021+021351'].loc[:,['HPS_name','OIII1_ID', 'OIII1_RA','OIII1_DEC','OII_Flux','OIII1_Flux']]
#print hps[hps['HPS_name'] == 'HPS123652+621125'].loc[:,['HPS_name','OIII1_ID', 'OIII1_RA','OIII1_DEC','OII_Flux','OIII1_Flux']]

    Unnamed: 0  Unnamed: 0.1          HPS_name  OII_ID  OII_z_x      OII_RA  \
18          18            18  HPS030639+000345    73.0   0.1073   46.664708   
25          25            25  HPS123633+621133   366.0   0.0787  189.137833   

      OII_DEC  OII_Flux  OII_EW  OIII1_ID  ...  OIII2_SN  Hb_SN  Hg_SN  Hd_SN  \
18   0.062500     437.0     1.7      77.0  ...      66.0   83.5   28.2    NaN   
25  62.192583      87.6    20.0     365.0  ...       6.1    7.5    NaN    NaN   

    NeIII1_SN  NeIII2_SN  OII_z_y  logM  logSFR  E(B-V)  
18       26.4        8.5      NaN   NaN     NaN     NaN  
25        NaN        NaN      NaN   NaN     NaN     NaN  

[2 rows x 63 columns]


In [9]:
#hps = hps.rename(columns={'OII_ID':'ID'})
#print hps

# hps_merge = hps_merge.rename(columns={'OII_ID':'ID'})
# print hps_merge

# Hb correction catalog

In [10]:
hb_corr = pd.read_csv('/Users/Briana/Documents/Grad_School/HPS/HPS_dataframes/hb_flux_correction.csv', delim_whitespace=True)
hb_corr['Flux_correction'] = hb_corr['Flux_correction']*1e16
print hb_corr

       ID  Flux_correction
0   363.0         0.799431
1   366.0         1.127675
2   375.0         9.520625
3   386.0         2.300704
4   413.0         3.735935
5   438.0         0.702793
6   449.0         1.442884
7   458.0         2.502626
8   158.0         0.727020
9   219.0         2.225950
10  225.0         1.291299
11  235.0         0.956961
12  237.0         0.623193
13  260.0         0.421522
14  278.0         2.085589
15  300.0         1.669884
16  303.0         0.406858
17  326.0         1.338706
18   35.0         8.911558
19   44.0         0.664095
20   65.0         8.693834
21   67.0         1.450334
22   73.0        41.579081
23  105.0         0.498073
24  118.0         0.680066
25  119.0         3.366373
26  125.0         7.120204
27  129.0         0.593160
28  138.0         1.894888


In [81]:
hb_corr = pd.read_csv('/Users/Briana/Documents/Grad_School/HPS/HPS_dataframes/hb_flux_correction.csv', delim_whitespace=True)
hb_corr['Flux_correction'] = hb_corr['Flux_correction']*1e16

# HPS objects with all lines

In [82]:
#convert units of lines to ergs/cm^2/s
hps['OII_e'] = hps['OII_Flux']/hps['OII_SN']
hps['Hb_e'] = hps['Hb_Flux']/hps['Hb_SN']
hps['OIII1_e'] = hps['OIII1_Flux']/hps['OIII1_SN']
hps['NeIII1_e'] = hps['NeIII1_Flux']/hps['NeIII1_SN']

hps['OII_Flux'] = hps['OII_Flux']*1e-20*1e7/10000*1e16
hps['Hb_Flux'] = hps['Hb_Flux']*1e-20*1e7/10000*1e16
hps['OIII1_Flux'] = hps['OIII1_Flux']*1e-20*1e7/10000*1e16
hps['NeIII1_Flux'] = hps['NeIII1_Flux']*1e-20*1e7/10000*1e16

hps['[OII]3727_e'] = hps['OII_e']*1e-20*1e7/10000*1e16
hps['[Hb]4861_e'] = hps['Hb_e']*1e-20*1e7/10000*1e16
hps['[OIII]5007_e'] = hps['OIII1_e']*1e-20*1e7/10000*1e16
hps['[NeIII]3870_e'] = hps['NeIII1_e']*1e-20*1e7/10000*1e16

#find objects with all 3 lines in HPS
hps_alllines = hps[~np.isnan(hps['OII_Flux']) & ~np.isnan(hps['Hb_Flux']) & ~np.isnan(hps['OIII1_Flux'])].loc[:,['HPS_name','OII_ID', 'OII_Flux', 'Hb_Flux', 'OIII1_Flux', 'NeIII1_Flux', '[OII]3727_e', '[Hb]4861_e', '[OIII]5007_e', '[NeIII]3870_e']]
hps_alllines['Source'] = 'HPS'
hps_alllines.rename(columns={'OII_ID':'ID', 'OII_Flux':'[OII]3727', 'Hb_Flux':'[Hb]4861', 'OIII1_Flux':'[OIII]5007', 'NeIII1_Flux':'[NeIII]3870'}, inplace=True)
print len(hps_alllines)
#hps_alllines

15


# HPS Objects OII Flux but missing other lines

In [83]:
#HPS objects with OII but missing other lines 
hps_misslines = hps[~np.isnan(hps['OII_Flux']) & (np.isnan(hps['Hb_Flux']) | np.isnan(hps['OIII1_Flux']))].loc[:,['HPS_name','OII_ID', 'OII_Flux','[OII]3727_e']]
hps_misslines.rename(columns={'OII_ID':'ID', 'OII_Flux':'[OII]_HPS', '[OII]3727_e': '[OII]_e_HPS'}, inplace=True)
print len(hps_misslines)
#hps_misslines

14


In [84]:
#HPS objects with OIII but missing other lines 
hps_misslines_oiii = hps[~np.isnan(hps['OIII1_Flux']) & np.isnan(hps['OII_Flux'])].loc[:,['HPS_name','OIII1_ID', 'OIII1_Flux','[OIII]5007_e',]]
hps_misslines_oiii.rename(columns={'OIII1_ID':'ID', 'OIII1_Flux':'[OIII]_HPS', '[OIII]5007_e': '[OIII]_e_HPS'}, inplace=True)
print len(hps_misslines_oiii)
hps_misslines_oiii

2


,HPS_name,ID,[OIII]_HPS,[OIII]_e_HPS
4,HPS100021+021351,234.0,2.44,0.173050
5,HPS123652+621125,430.0,0.66,0.111864


# Observed HPS object Fluxes with LRS2

In [85]:
#lines = pd.read_csv('/Volumes/Briana_mac3/HPS/HPS_dataframes/hps_lrs2_fits_newInt.csv')

#list with new fits to NeIII in LRS2 spectra
lines = pd.read_csv('/Volumes/Bri_USB/hps_lrs2_fits_wNe_4.csv')
lines = lines[lines['HPS_name'] != 'HPS123652+621125']
lines = lines[lines['HPS_name'] != 'HPS100021+021351']
print len(lines)

OIII_lines = pd.read_csv('/Users/Briana/Documents/Grad_School/HPS/HPS_dataframes/hps_lsr2_fits_wNe_OIIIobs.csv')

#the [OIII]5007 line fit in my code is actually 5007+4959 so correct it to just 5007 
#lines['[OIII]5007'] = lines['[OIII]5007']/(1+(1/2.98))

#remove row 35 because no signal and row 25, 37 because bad data
lines = lines.drop([37], axis=0)

lines = lines.loc[:, ~lines.columns.str.contains('^Unnamed')]
OIII_lines = OIII_lines.loc[:, ~OIII_lines.columns.str.contains('^Unnamed')]

lines = lines.append(OIII_lines, sort=True)

lines['[Ha]6562_e'].fillna('[Nan, Nan]', inplace=True)
lines['[NII]6583_e'].fillna('[Nan, Nan]', inplace=True)

print lines

# 'HPS100021+021351' 24,25, 34,35
# 'HPS123652+621125' 38,39
#print lines.loc[:,['HPS_name', '[OII]3726', '[OII]3729']]
#print lines.loc[:,['[OII]3726_e','[OII]3729_e', '[Hb]4861_e', '[OIII]5007_e']]

37
            HPS_name                Obs_ID  [Ha]6562      [Ha]6562_e  \
0   HPS123632+621037  20170513_obs10_exp01       NaN      [Nan, Nan]   
1   HPS123632+621037  20170513_obs10_exp02       NaN      [Nan, Nan]   
2   HPS123632+621037  20170518_obs14_exp01       NaN      [Nan, Nan]   
3   HPS123656+621420  20170524_obs10_exp01       NaN      [Nan, Nan]   
4   HPS123656+621420  20170524_obs10_exp02       NaN      [Nan, Nan]   
5   HPS030655-000050  20170918_obs18_exp01       NaN      [Nan, Nan]   
6   HPS030651-000234  20170922_obs17_exp01       NaN      [Nan, Nan]   
7   HPS030651-000234  20170922_obs17_exp02       NaN      [Nan, Nan]   
8   HPS030655-000050  20170923_obs21_exp01       NaN      [Nan, Nan]   
9   HPS030652+000123  20171012_obs20_exp01       NaN      [Nan, Nan]   
10  HPS030652+000123  20171012_obs20_exp02       NaN      [Nan, Nan]   
11  HPS030655-000050  20171013_obs16_exp01       NaN      [Nan, Nan]   
12  HPS030655-000050  20171013_obs16_exp02       NaN      [Na

In [86]:
# lines_new = pd.read_csv('/Volumes/Briana_mac3/HPS/HPS_dataframes/hps_lrs2_fits_newInt.csv')
# lines_new.rename(columns={'[OIII]5007':'[OIII]5007+4959'}, inplace=True)
# #remove row 35 because no signal and row 25 because bad data
# lines_new = lines_new.drop([25,35], axis=0)

# lines_1 = lines.loc[:,['HPS_name','Obs_ID','[OII]3726','[OII]3729', '[Hb]4861', '[OIII]5007+4959']]
# lines_1['[OII]3727'] = lines_1['[OII]3726'] + lines_1['[OII]3729']
# lines_1 = lines_1.loc[:,['HPS_name','Obs_ID','[OII]3727', '[Hb]4861', '[OIII]5007+4959']]
# lines_1.to_csv('/Volumes/Briana_mac3/HPS/LRS2_reduction_greg/hps_lrs2_quad_fluxes.csv')


# lines_2 = lines_new.loc[:,['HPS_name','Obs_ID','[OII]3726','[OII]3729', '[Hb]4861', '[OIII]5007+4959']]
# lines_2['[OII]3727'] = np.add(lines_2['[OII]3726'], lines_2['[OII]3729'])
# lines_2 = lines_2.loc[:,['HPS_name','Obs_ID','[OII]3727', '[Hb]4861', '[OIII]5007+4959']]
# lines_2.to_csv('/Volumes/Briana_mac3/HPS/LRS2_reduction_greg/hps_lrs2_amp_fluxes.csv')

# lines_3 = lines_2
# lines_3['[OII]3727'] = (lines_2['[OII]3727'] - lines_1['[OII]3727'])/lines_1['[OII]3727']
# lines_3['[OIII]5007+4959'] = (lines_2['[OIII]5007+4959'] - lines_1['[OIII]5007+4959'])/lines_1['[OIII]5007+4959']
# lines_3['[Hb]4861'] = (lines_2['[Hb]4861'] - lines_1['[Hb]4861'])/lines_1['[Hb]4861']

# print lines_3.describe()
# print lines_1, 



In [87]:

#average up and lo errors
lines['[OII]3726_e'] = [np.round(np.average([float(lines['[OII]3726_e'].iloc[i].split(',')[0][1::]), float(lines['[OII]3726_e'].iloc[i].split(',')[1][0:-1])]), 3) for i in range(len(lines))]
lines['[OII]3729_e'] = [np.round(np.average([float(lines['[OII]3729_e'].iloc[i].split(',')[0][1::]), float(lines['[OII]3729_e'].iloc[i].split(',')[1][0:-1])]), 3) for i in range(len(lines))]
lines['[Hb]4861_e'] = [np.round(np.average([float(lines['[Hb]4861_e'].iloc[i].split(',')[0][1::]), float(lines['[Hb]4861_e'].iloc[i].split(',')[1][0:-1])]), 3) for i in range(len(lines))]
#lines['[OIII]5007_e'] = [np.round(np.average([float(lines['[OIII]5007_e'].iloc[i].split(',')[0][1::])/(1+(1/2.98)), float(lines['[OIII]5007_e'].iloc[i].split(',')[1][0:-1])/(1+(1/2.98))]), 3) for i in range(len(lines))]
lines['[OIII]5007_e'] = [np.round(np.average([float(lines['[OIII]5007_e'].iloc[i].split(',')[0][1::]), float(lines['[OIII]5007_e'].iloc[i].split(',')[1][0:-1])]), 3) for i in range(len(lines))]

lines['[NeIII]3870_e'] = [np.round(np.average([float(lines['[NeIII]3870_e'].iloc[i].split(',')[0][1::]), float(lines['[NeIII]3870_e'].iloc[i].split(',')[1][0:-1])]), 3) for i in range(len(lines))]
lines['[Ha]6562_e'] = [np.round(np.average([float(lines['[Ha]6562_e'].iloc[i].split(',')[0][1::]), float(lines['[Ha]6562_e'].iloc[i].split(',')[1][0:-1])]), 3) for i in range(len(lines))]
lines['[NII]6583_e'] = [np.round(np.average([float(lines['[NII]6583_e'].iloc[i].split(',')[0][1::]), float(lines['[NII]6583_e'].iloc[i].split(',')[1][0:-1])]), 3) for i in range(len(lines))]

In [88]:
#add OII lines, propogate OII errors 
lines['[OII]3727'] = lines['[OII]3726'] + lines['[OII]3729']
lines['[OII]3727_e'] =  np.round(np.sqrt((lines['[OII]3726_e']**2) + (lines['[OII]3729_e']**2)), 3)

#print lines.loc[:,['HPS_name', 'Obs_ID', '[OII]3727', '[Hb]4861', '[OIII]5007','[OII]3727_e', '[Hb]4861_e', '[OIII]5007_e']]

In [89]:
line_groups = lines.groupby('HPS_name')['[OII]3727', '[NeIII]3870', '[Hb]4861', '[OIII]5007', '[Ha]6562', '[NII]6583', '[OII]3727_e', '[NeIII]3870_e', '[Hb]4861_e', '[OIII]5007_e', '[Ha]6562_e', '[NII]6583_e'].mean()
line_groups['Source'] = 'LRS2'

line_groups['HPS_name'] = line_groups.index
print len(line_groups)
print line_groups

16
                  [OII]3727  [NeIII]3870  [Hb]4861  [OIII]5007  [Ha]6562  \
HPS_name                                                                   
HPS030630+000128   3.082500     0.262000  0.757000    2.155000       NaN   
HPS030638+000240   1.115000    -0.068250  0.450500    0.017750       NaN   
HPS030651-000234   1.547000     0.089000  0.271000    0.638500       NaN   
HPS030652+000123   6.559500     0.368500  2.139000    4.322000       NaN   
HPS030655+000213   1.943000     0.403500  1.084500    2.939500       NaN   
HPS030655-000050   2.640250     0.271750  1.033000    1.768750       NaN   
HPS100008+021542   2.153250     0.064250  0.419250    1.100000       NaN   
HPS100021+021223   0.837500     0.059500  0.068000    0.309000       NaN   
HPS100021+021237  -0.223000    -0.037500 -0.158500   -0.135500       NaN   
HPS100021+021351   1.041000     0.449000  0.403000    2.508000     1.468   
HPS100028+021858   1.554500     0.090500  0.329500    0.837750       NaN   
HPS100037

In [90]:
#normalize observed fluxes to HPS OII fluxes 
obs_line_merge = pd.merge(line_groups, hps_misslines, on='HPS_name', how='outer')
obs_line_merge_oiii = pd.merge(line_groups, hps_misslines_oiii, on='HPS_name', how='outer')
#print obs_line_merge

In [91]:
#list observations that are missing and HPS OII detection
OIII_obs_norm = obs_line_merge_oiii[~np.isnan(obs_line_merge_oiii['[OIII]_HPS'])]
print len(OIII_obs_norm)
OIII_obs_norm

2


,[OII]3727,[NeIII]3870,[Hb]4861,[OIII]5007,[Ha]6562,[NII]6583,[OII]3727_e,[NeIII]3870_e,[Hb]4861_e,[OIII]5007_e,[Ha]6562_e,[NII]6583_e,Source,HPS_name,ID,[OIII]_HPS,[OIII]_e_HPS
9,1.041,0.449,0.403,2.508,1.468,0.076,0.706,0.402,0.180,0.282,0.156,0.060,LRS2,HPS100021+021351,234.0,2.44,0.173050
14,0.555,0.009,0.088,0.406,0.743,0.009,0.297,0.112,0.088,0.127,0.078,0.027,LRS2,HPS123652+621125,430.0,0.66,0.111864


In [92]:
OII_obs_norm = obs_line_merge[~np.isnan(obs_line_merge['[OII]_HPS'])]
print len(OII_obs_norm)
#OII_obs_norm.to_csv('/Volumes/Briana_mac3/HPS/HPS_dataframes/pre_norm_OII_lrs2.csv')
#OII_obs_norm

14


In [93]:
#propogate LRS2 observation errors for normalized values
norm_Hb_e   = OII_obs_norm['[Hb]4861']*np.sqrt((np.divide(OII_obs_norm['[Hb]4861_e'], OII_obs_norm['[Hb]4861'])**2)+(np.divide(OII_obs_norm['[OII]3727_e'], OII_obs_norm['[OII]3727'])**2)+(np.divide(OII_obs_norm['[OII]_e_HPS'], OII_obs_norm['[OII]_HPS'])**2))

OII_obs_norm['[Hb]4861_e'] = norm_Hb_e
OII_obs_norm['[OIII]5007_e'] = OII_obs_norm['[OIII]5007_e']*(OII_obs_norm['[OII]_HPS']/OII_obs_norm['[OII]3727'])
OII_obs_norm['[OII]3727_e']  = OII_obs_norm['[OII]3727_e']*(OII_obs_norm['[OII]_HPS']/OII_obs_norm['[OII]3727'])
OII_obs_norm['[NeIII]3870_e']  = OII_obs_norm['[NeIII]3870_e']*(OII_obs_norm['[OII]_HPS']/OII_obs_norm['[OII]3727'])

#normalize LRS2 observations
norm_OIII  = (OII_obs_norm['[OIII]5007']/OII_obs_norm['[OII]3727'])*OII_obs_norm['[OII]_HPS']
norm_Hb    = (OII_obs_norm['[Hb]4861']/OII_obs_norm['[OII]3727'])*OII_obs_norm['[OII]_HPS']
norm_NeIII = (OII_obs_norm['[NeIII]3870']/OII_obs_norm['[OII]3727'])*OII_obs_norm['[OII]_HPS']

OII_obs_norm['[OIII]5007']  = norm_OIII
OII_obs_norm['[Hb]4861'] = norm_Hb
OII_obs_norm['[NeIII]3870'] = norm_NeIII
OII_obs_norm['[OII]3727']  = OII_obs_norm['[OII]_HPS']

print len(OII_obs_norm)
#print OII_obs_norm

14


/Users/Briana/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/Briana/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/Briana/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [94]:
#propogate LRS2 observation errors for normalized values
norm_Hb_e_oiii = OIII_obs_norm['[Hb]4861']*np.sqrt((np.divide(OIII_obs_norm['[Hb]4861_e'], OIII_obs_norm['[Hb]4861'])**2)+(np.divide(OIII_obs_norm['[OIII]5007_e'], OIII_obs_norm['[OIII]5007'])**2)+(np.divide(OIII_obs_norm['[OIII]_e_HPS'], OIII_obs_norm['[OIII]_HPS'])**2))

OIII_obs_norm['[Hb]4861_e'] = norm_Hb_e_oiii
OIII_obs_norm['[OIII]5007_e'] = OIII_obs_norm['[OIII]5007_e']*(OIII_obs_norm['[OIII]_HPS']/OIII_obs_norm['[OIII]5007'])
OIII_obs_norm['[OII]3727_e']  = OIII_obs_norm['[OII]3727_e']*(OIII_obs_norm['[OIII]_HPS']/OIII_obs_norm['[OIII]5007'])
OIII_obs_norm['[NeIII]3870_e']  = OIII_obs_norm['[NeIII]3870_e']*(OIII_obs_norm['[OIII]_HPS']/OIII_obs_norm['[OIII]5007'])
OIII_obs_norm['[Ha]6562_e']  = OIII_obs_norm['[Ha]6562_e']*(OIII_obs_norm['[OIII]_HPS']/OIII_obs_norm['[OIII]5007'])
OIII_obs_norm['[NII]6583_e']  = OIII_obs_norm['[NII]6583_e']*(OIII_obs_norm['[OIII]_HPS']/OIII_obs_norm['[OIII]5007'])

#normalize LRS2 observations
norm_OII_oiii   = (OIII_obs_norm['[OII]3727']/OIII_obs_norm['[OIII]5007'])*OIII_obs_norm['[OIII]_HPS']
norm_Hb_oiii    = (OIII_obs_norm['[Hb]4861']/OIII_obs_norm['[OIII]5007'])*OIII_obs_norm['[OIII]_HPS']
norm_NeIII_oiii = (OIII_obs_norm['[NeIII]3870']/OIII_obs_norm['[OIII]5007'])*OIII_obs_norm['[OIII]_HPS']
norm_Ha_oiii = (OIII_obs_norm['[Ha]6562']/OIII_obs_norm['[OIII]5007'])*OIII_obs_norm['[OIII]_HPS']
norm_NII_oiii = (OIII_obs_norm['[NII]6583']/OIII_obs_norm['[OIII]5007'])*OIII_obs_norm['[OIII]_HPS']

OIII_obs_norm['[OII]3727']  = norm_OII_oiii
OIII_obs_norm['[Hb]4861'] = norm_Hb_oiii
OIII_obs_norm['[Ne]3870'] = norm_NeIII_oiii
OIII_obs_norm['[Ha]6562'] = norm_Ha_oiii
OIII_obs_norm['[NII]6583'] = norm_NII_oiii
OIII_obs_norm['[OIII]5007']  = OIII_obs_norm['[OIII]_HPS']

print OIII_obs_norm
OIII_obs_norm.to_csv('/Users/Briana/Documents/Grad_School/HPS/HPS_dataframes/OIII_norm_fluxes.csv')


/Users/Briana/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/Briana/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/Briana/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

    [OII]3727  [NeIII]3870  [Hb]4861  [OIII]5007  [Ha]6562  [NII]6583  \
9    1.012775        0.449  0.392073        2.44  1.428198   0.073939   
14   0.902217        0.009  0.143054        0.66  1.207833   0.014631   

    [OII]3727_e  [NeIII]3870_e  [Hb]4861_e  [OIII]5007_e  [Ha]6562_e  \
9      0.686858       0.391100    0.187804      0.274354    0.151770   
14     0.482808       0.182069    0.093403      0.206453    0.126798   

    [NII]6583_e Source          HPS_name     ID  [OIII]_HPS  [OIII]_e_HPS  \
9      0.058373   LRS2  HPS100021+021351  234.0        2.44      0.173050   
14     0.043892   LRS2  HPS123652+621125  430.0        0.66      0.111864   

    [Ne]3870  
9   0.436826  
14  0.014631  


/Users/Briana/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Briana/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Briana/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/

# Merge of HPS and LRS2 fluxes

In [95]:
# lines_mg = pd.merge(line_groups, hps_merge, on='HPS_name', how='left')
# print lines_mg

In [96]:
frames = [hps_alllines, line_groups]
obs_tot = pd.concat(frames, sort=True)
obs_tot.set_index(np.arange(len(obs_tot)), inplace=True)
print obs_tot

            HPS_name     ID Source  [Ha]6562  [Ha]6562_e    [Hb]4861  \
0   HPS030638+000015   65.0    HPS       NaN         NaN    2.480000   
1   HPS022127-043019   35.0    HPS       NaN         NaN   29.090000   
2   HPS030639+000345   73.0    HPS       NaN         NaN  126.450000   
3   HPS030649+000314  105.0    HPS       NaN         NaN    2.370000   
4   HPS030657+000139  138.0    HPS       NaN         NaN    3.360000   
5   HPS100018+021818  219.0    HPS       NaN         NaN    4.030000   
6   HPS100018+021426  225.0    HPS       NaN         NaN    1.180000   
7   HPS100032+021356  278.0    HPS       NaN         NaN    3.680000   
8   HPS100045+021823  326.0    HPS       NaN         NaN    1.940000   
9   HPS123633+621133  366.0    HPS       NaN         NaN    1.930000   
10  HPS123636+621135  375.0    HPS       NaN         NaN   14.130000   
11  HPS123641+621131  386.0    HPS       NaN         NaN    4.030000   
12  HPS123648+621426  413.0    HPS       NaN         NaN    4.36

In [97]:
#concat the HPS sources with the LRS2 sources 
frames_norm = [hps_alllines, OII_obs_norm]
obs_tot_norm = pd.concat(frames_norm, sort=True)
obs_tot_norm.set_index(np.arange(len(obs_tot_norm)), inplace=True)
print obs_tot_norm
print len(obs_tot_norm)

            HPS_name     ID Source  [Ha]6562  [Ha]6562_e    [Hb]4861  \
0   HPS030638+000015   65.0    HPS       NaN         NaN    2.480000   
1   HPS022127-043019   35.0    HPS       NaN         NaN   29.090000   
2   HPS030639+000345   73.0    HPS       NaN         NaN  126.450000   
3   HPS030649+000314  105.0    HPS       NaN         NaN    2.370000   
4   HPS030657+000139  138.0    HPS       NaN         NaN    3.360000   
5   HPS100018+021818  219.0    HPS       NaN         NaN    4.030000   
6   HPS100018+021426  225.0    HPS       NaN         NaN    1.180000   
7   HPS100032+021356  278.0    HPS       NaN         NaN    3.680000   
8   HPS100045+021823  326.0    HPS       NaN         NaN    1.940000   
9   HPS123633+621133  366.0    HPS       NaN         NaN    1.930000   
10  HPS123636+621135  375.0    HPS       NaN         NaN   14.130000   
11  HPS123641+621131  386.0    HPS       NaN         NaN    4.030000   
12  HPS123648+621426  413.0    HPS       NaN         NaN    4.36

# Final HPS OII Flux Catalog

In [98]:
# obs_tot_wHb = pd.merge(obs_tot, hb_corr, on='ID', how='left')
# obs_tot_wHb = obs_tot_wHb.rename(columns={'Flux_correction':'Hb_absorption'})
# obs_tot_wHb = obs_tot_wHb.drop([24,29], axis=0)
# print obs_tot_wHb

In [99]:
#obs_tot_wHb.to_csv('/Volumes/Briana_mac3/HPS/HPS_dataframes/HPS_OII_Fluxes.csv')

In [100]:
# obs_tot_Table = Table.from_pandas(obs_tot_wHb)
# obs_tot_Table['[OII]3727'] = np.round(obs_tot_Table['[OII]3727'],3)
# obs_tot_Table['[OII]3727_e'] = np.round(obs_tot_Table['[OII]3727_e'],3)
# obs_tot_Table['[NeIII]3870'] = np.round(obs_tot_Table['[NeIII]3870'],3)
# obs_tot_Table['[NeIII]3870_e'] = np.round(obs_tot_Table['[NeIII]3870_e'],3)
# obs_tot_Table['[OIII]5007+4959'] = np.round(obs_tot_Table['[OIII]5007+4959'],3)
# obs_tot_Table['[OIII]_e'] = np.round(obs_tot_Table['[OIII]_e'],3)
# obs_tot_Table['[Hb]4861'] = np.round(obs_tot_Table['[Hb]4861'],3)
# obs_tot_Table['[Hb]4861_e'] = np.round(obs_tot_Table['[Hb]4861_e'],3)
# obs_tot_Table['Hb_absorption'] = np.round(obs_tot_Table['Hb_absorption'],3)

# obs_tot_Table.write('/Volumes/Briana_mac3/HPS/HPS_dataframes/HPS_OII_Fluxes.tex', format='ascii.aastex', overwrite=True)
# print obs_tot_Table

In [101]:
#add Hb absorption and save file for normalized LRS2 fluxes 
obs_tot_norm_wHb = pd.merge(obs_tot_norm, hb_corr, on='ID', how='left')
obs_tot_norm_wHb = obs_tot_norm_wHb.rename(columns={'Flux_correction':'Hb_absorption'})
obs_tot_norm_wHb.drop(['[OII]_HPS', '[OII]_e_HPS'], axis=1, inplace=True)
#obs_tot_norm_wHb = obs_tot_norm_wHb.drop([24,29], axis=0)
print obs_tot_norm_wHb.loc[:, ['HPS_name', '[OII]3727']]
obs_tot_norm_wHb.to_csv('/Users/Briana/Documents/Grad_School/HPS/HPS_dataframes/OII_norm_fluxes.csv')

            HPS_name  [OII]3727
0   HPS030638+000015      24.38
1   HPS022127-043019      75.21
2   HPS030639+000345      43.70
3   HPS030649+000314       5.66
4   HPS030657+000139       9.42
5   HPS100018+021818      18.55
6   HPS100018+021426       5.46
7   HPS100032+021356      11.81
8   HPS100045+021823       7.58
9   HPS123633+621133       8.76
10  HPS123636+621135      40.77
11  HPS123641+621131      15.34
12  HPS123648+621426      19.77
13  HPS123659+621404       6.59
14  HPS123702+621123      11.43
15  HPS030630+000128       4.37
16  HPS030638+000240       4.57
17  HPS030651-000234       4.03
18  HPS030652+000123      20.76
19  HPS030655+000213       5.08
20  HPS030655-000050      22.39
21  HPS100008+021542       4.53
22  HPS100021+021223       3.55
23  HPS100021+021237       1.94
24  HPS100028+021858       2.27
25  HPS100037+021254       4.18
26  HPS100039+021246       3.11
27  HPS123632+621037       6.18
28  HPS123656+621420       3.04


In [102]:
source_group = obs_tot_norm_wHb.groupby('Source')
lrs2_gp = source_group.get_group('LRS2')
hps_gp  = source_group.get_group('HPS')
print len(lrs2_gp), len(hps_gp)

14 15


In [103]:
#this is for the OIII objects
#add Hb absorption and save file for normalized LRS2 fluxes 
OIII_obs_norm['Hb_absorption'] = [0.12, 0.3]
#OIII_obs_norm['[NeIII]3870_e'] = OIII_obs_norm['[NeIII]3870']
#OIII_obs_norm.drop(['[OIII]_HPS', '[OIII]_e_HPS'], axis=1, inplace=True)
print OIII_obs_norm
OIII_obs_norm.to_csv('/Users/Briana/Documents/Grad_School/HPS/HPS_dataframes/OIII_norm_fluxes.csv')

    [OII]3727  [NeIII]3870  [Hb]4861  [OIII]5007  [Ha]6562  [NII]6583  \
9    1.012775        0.449  0.392073        2.44  1.428198   0.073939   
14   0.902217        0.009  0.143054        0.66  1.207833   0.014631   

    [OII]3727_e  [NeIII]3870_e  [Hb]4861_e  [OIII]5007_e  [Ha]6562_e  \
9      0.686858       0.391100    0.187804      0.274354    0.151770   
14     0.482808       0.182069    0.093403      0.206453    0.126798   

    [NII]6583_e Source          HPS_name     ID  [OIII]_HPS  [OIII]_e_HPS  \
9      0.058373   LRS2  HPS100021+021351  234.0        2.44      0.173050   
14     0.043892   LRS2  HPS123652+621125  430.0        0.66      0.111864   

    [Ne]3870  Hb_absorption  
9   0.436826           0.12  
14  0.014631           0.30  


/Users/Briana/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [104]:
obs_tot_Table = Table.from_pandas(obs_tot_norm_wHb)
obs_tot_Table['[OII]3727'] = np.round(obs_tot_Table['[OII]3727'],3)
obs_tot_Table['[OII]3727_e'] = np.round(obs_tot_Table['[OII]3727_e'],3)
obs_tot_Table['[NeIII]3870'] = np.round(obs_tot_Table['[NeIII]3870'],3)
obs_tot_Table['[NeIII]3870_e'] = np.round(obs_tot_Table['[NeIII]3870_e'],3)
obs_tot_Table['[OIII]5007'] = np.round(obs_tot_Table['[OIII]5007'],3)
obs_tot_Table['[OIII]5007_e'] = np.round(obs_tot_Table['[OIII]5007_e'],3)
obs_tot_Table['[Hb]4861'] = np.round(obs_tot_Table['[Hb]4861'],3)
obs_tot_Table['[Hb]4861_e'] = np.round(obs_tot_Table['[Hb]4861_e'],3)
obs_tot_Table['Hb_absorption'] = np.round(obs_tot_Table['Hb_absorption'],3)

obs_tot_Table.write('/Users/Briana/Documents/Grad_School/HPS/HPS_dataframes/HPS_OII_norm_Fluxes.tex', format='ascii.aastex', overwrite=True)
print obs_tot_Table

    HPS_name       ID  Source [Ha]6562 ... [OII]3727 [OII]3727_e Hb_absorption
---------------- ----- ------ -------- ... --------- ----------- -------------
HPS030638+000015  65.0    HPS       -- ...     24.38       1.037         8.694
HPS022127-043019  35.0    HPS       -- ...     75.21       0.909         8.912
HPS030639+000345  73.0    HPS       -- ...      43.7       1.201        41.579
HPS030649+000314 105.0    HPS       -- ...      5.66        0.51         0.498
HPS030657+000139 138.0    HPS       -- ...      9.42       0.335         1.895
HPS100018+021818 219.0    HPS       -- ...     18.55        0.42         2.226
HPS100018+021426 225.0    HPS       -- ...      5.46        0.25         1.291
HPS100032+021356 278.0    HPS       -- ...     11.81       0.305         2.086
HPS100045+021823 326.0    HPS       -- ...      7.58       0.387         1.339
HPS123633+621133 366.0    HPS       -- ...      8.76       0.596         1.128
             ...   ...    ...      ... ...       ...

# Pandas Testing

In [105]:
#pos = SkyCoord('0h8m05.63s +14d50m23.3s', frame='icrs')
#xid = SDSS.query_region(pos, spectro=True)
#print(xid)

In [70]:
#miss_OIII =  hps['HPS_name'].loc[np.isnan(hps['OIII1_Flux'])]
#miss_Hb   = hps['HPS_name'].loc[np.isnan(hps['Hb_Flux'])]

#miss = hps['HPS_name'].loc[np.isnan(hps['OIII1_Flux']) | np.isnan(hps['Hb_Flux'])]
#line = hps['HPS_name'].loc[~np.isnan(hps['OIII1_Flux']) & ~np.isnan(hps['Hb_Flux'])]
#print miss
#print line

ra  = hps[hps['HPS_name'] == 'HPS030638+000240']['OII_RA']
dec = hps[hps['HPS_name'] == 'HPS030638+000240']['OII_DEC']
c = SkyCoord(ra, dec, unit='deg', frame='icrs')
print c.ra.hms
print c.dec

hms_tuple(h=array([3.]), m=array([6.]), s=array([38.91]))
[u'-0d02m40.1s']


In [71]:
hps_all_lines = hps.dropna(subset=['OII_Flux','OIII1_Flux', 'Hb_Flux'])
hps_all_lines['OII_Flux'].iloc[0]

24.380000000000003

In [72]:
hps['OII_Flux'].loc[hps.HPS_name == 'HPS030638+000240']

0    4.57
Name: OII_Flux, dtype: float64

In [78]:
def find_SN(row):
    if np.isnan(row['NeIII2_ID']):
        return np.nan
    else:
        sn = hps_full.loc[hps_full['HPS_ID'] == row['NeIII2_ID']]['S_N'].values[0]
        return sn

#You give apply the function and the argument axis=1 to it feeds the function the rows of the dataframe
#It stores these new values in a new column called SN
hps['NeIII2_SN'] = hps.apply(find_SN, axis=1)


In [80]:
print hps.columns
hps.to_csv('/Volumes/Briana_mac3/HPS/HPS_dataframes/HPS_cat_table.dat')

Index([u'HPS_name', u'OII_ID', u'OII_z', u'OII_RA', u'OII_DEC', u'OII_Flux',
       u'OII_EW', u'OIII1_ID', u'OIII1_z', u'OIII1_RA', u'OIII1_DEC',
       u'OIII1_Flux', u'OIII1_EW', u'OIII2_ID', u'OIII2_z', u'OIII2_RA',
       u'OIII2_DEC', u'OIII2_Flux', u'OIII2_EW', u'Hb_ID', u'Hb_z', u'Hb_RA',
       u'Hb_DEC', u'Hb_Flux', u'Hb_EW', u'Hg_ID', u'Hg_z', u'Hg_RA', u'Hg_DEC',
       u'Hg_Flux', u'Hg_EW', u'Hd_ID', u'Hd_z', u'Hd_RA', u'Hd_DEC',
       u'Hd_Flux', u'Hd_EW', u'NeIII1_ID', u'NeIII1_z', u'NeIII1_RA',
       u'NeIII1_DEC', u'NeIII1_Flux', u'NeIII1_EW', u'NeIII2_ID', u'NeIII2_z',
       u'NeIII2_RA', u'NeIII2_DEC', u'NeIII2_Flux', u'NeIII2_EW', u'OII_SN',
       u'OIII1_SN', u'OIII2_SN', u'Hb_SN', u'Hg_SN', u'Hd_SN', u'NeIII1_SN',
       u'NeIII2_SN'],
      dtype='object')


# LRS2 Observations Catalog

In [25]:
lrs2_obs = pd.read_csv('/Volumes/Briana_mac3/HPS/HPS_dataframes/HPS_lrs2Obs_table.dat')
np.shape(lrs2_obs)

(22, 6)

In [28]:
redux = [[0], [1], [1], [0], [1], [1,2], [1,2], [0], [0], [0], [0], [0], [0], [1], [0], [0], [0], [0], [0], [0], [2], [0]]

In [29]:
lrs2_obs['good_exposures'] = redux
lrs2_obs

,Unnamed: 0,HPS_name,obs_date,standard,obj_vis,good_exposures
0,0,HPS030630+000128,20170913,BD_+17_4708_056_E,N,[0]
1,1,HPS030651-000234,20170922,GD248_056_E,Y,[1]
2,2,HPS030652+000123,20171012,HD_19445_056_E,Y,[1]
3,3,HPS030655+000213,20171014,GD248_056_E,Y,[0]
4,4,HPS030655-000050,20171013,G191B2B_056_E,Y,[1]
5,5,HPS100008+021542,20171213,HD_19445_056_W,Y,"[1, 2]"
6,6,HPS100008+021542,20171119,HZ_4_056_W,Y,"[1, 2]"
7,7,HPS100021+021223,20171129,FEIGE_66_056_E,Y,[0]
8,8,HPS100021+021237,20171117,HD_19445_056_W,Y,[0]
9,9,HPS100021+021351,20171128,HD_19445_056_E,Y,[0]


In [18]:
row_1210 = ['HPS030630+000128', 20171210, 'HILTNER_102_056_W', 'Y', [2]]
row_0417 = ['HPS123652+621125', 20180417, 'FEIGE_66_056_W', 'M', [0]]
lrs2_obs.loc[20] = row_1210
lrs2_obs.loc[21] = row_0417
lrs2_obs

,HPS_name,obs_date,standard,obj_vis,good_exposures
0,HPS030630+000128,20170913,BD_+17_4708_056_E,N,0
1,HPS030651-000234,20170922,GD248_056_E,Y,1
2,HPS030652+000123,20171012,HD_19445_056_E,Y,1
3,HPS030655+000213,20171014,GD248_056_E,Y,0
4,HPS030655-000050,20171013,G191B2B_056_E,Y,1
5,HPS100008+021542,20171213,HD_19445_056_W,Y,"(1, 2)"
6,HPS100008+021542,20171119,HZ_4_056_W,Y,"(1, 2)"
7,HPS100021+021223,20171129,FEIGE_66_056_E,Y,0
8,HPS100021+021237,20171117,HD_19445_056_W,Y,0
9,HPS100021+021351,20171128,HD_19445_056_E,Y,0


In [30]:
lrs2_obs.to_csv('/Volumes/Briana_mac3/HPS/HPS_dataframes/HPS_lrs2Obs_table.dat')

In [6]:
lrs2_obs.sort_values('obs_date')

,HPS_name,obs_date,standard,obj_vis
16,HPS123632+621037,20170513,BD_+26_2606,None
15,HPS123617+621418,20170513,BD_+26_2606,None
17,HPS123617+621418,20170518,HZ_44-B_056_W,None
18,HPS123632+621037,20170518,HZ_44-B_056_W,None
19,HPS123656+621420,20170524,BD_+26_2606_056_W,Y
0,HPS030630+000128,20170913,BD_+17_4708_056_E,N
1,HPS030651-000234,20170922,GD248_056_E,Y
2,HPS030652+000123,20171012,HD_19445_056_E,Y
4,HPS030655-000050,20171013,G191B2B_056_E,Y
3,HPS030655+000213,20171014,GD248_056_E,Y


In [27]:
bridge = pd.read_csv('/Volumes/Briana_mac3/HPS/HPS_dataframes/HPS_bridge_table.ascii', delim_whitespace=True)
bridge

,HPS_name,OII_z,logM,logSFR,E(B-V)
0,HPS022127-043019,0.0841,9.765,0.4603,0.223
1,HPS030630+000128,0.1191,8.740,-0.7672,0.089
2,HPS030638+000015,0.1121,10.963,0.8933,0.503
3,HPS030638+000240,0.1123,9.670,-0.5375,0.202
4,HPS030649+000314,0.1094,7.554,-0.7358,0.088
5,HPS030651-000234,0.1072,9.009,-0.8667,0.103
6,HPS030652+000123,0.1121,9.519,0.0504,0.173
7,HPS030655-000050,0.1118,10.284,0.3778,0.300
8,HPS030655+000213,0.1377,8.753,-0.6231,0.064
9,HPS030657+000139,0.1027,9.130,-0.7505,0.012


In [32]:
HPS_OII = pd.merge(hps, bridge, how='left', on=['HPS_name'])
HPS_OII.columns

#HPS_OII.to_csv('HPS_cat_table.dat')

Index([u'Unnamed: 0', u'HPS_name', u'OII_ID', u'OII_z_x', u'OII_RA',
       u'OII_DEC', u'OII_Flux', u'OII_EW', u'OIII1_ID', u'OIII1_z',
       u'OIII1_RA', u'OIII1_DEC', u'OIII1_Flux', u'OIII1_EW', u'OIII2_ID',
       u'OIII2_z', u'OIII2_RA', u'OIII2_DEC', u'OIII2_Flux', u'OIII2_EW',
       u'Hb_ID', u'Hb_z', u'Hb_RA', u'Hb_DEC', u'Hb_Flux', u'Hb_EW', u'Hg_ID',
       u'Hg_z', u'Hg_RA', u'Hg_DEC', u'Hg_Flux', u'Hg_EW', u'Hd_ID', u'Hd_z',
       u'Hd_RA', u'Hd_DEC', u'Hd_Flux', u'Hd_EW', u'NeIII1_ID', u'NeIII1_z',
       u'NeIII1_RA', u'NeIII1_DEC', u'NeIII1_Flux', u'NeIII1_EW', u'NeIII2_ID',
       u'NeIII2_z', u'NeIII2_RA', u'NeIII2_DEC', u'NeIII2_Flux', u'NeIII2_EW',
       u'OII_SN', u'OIII1_SN', u'OIII2_SN', u'Hb_SN', u'Hg_SN', u'Hd_SN',
       u'NeIII1_SN', u'NeIII2_SN', u'OII_z_y', u'logM', u'logSFR', u'E(B-V)'],
      dtype='object')

In [7]:
#add a column for the default redshift (z)
#z = OII_z if there is a values else z=OIII1_z

def build_z(row):
    if np.isnan(row.OII_z):
        z = row.OIII1_z
    else:
        z = row.OII_z
    return z

hps['z'] = hps.apply(build_z, axis=1)

In [8]:
hetdex_samp = hps.query('z < 0.1')

In [9]:
#add a column for the default coorinates (coords)
#they are written as astropy coordinates
#coords = OII RA and DEC if there is are values else it useds OIII1 RA and DEC

def build_astropy_coords(row):
    if np.isnan(row.OII_z):
        line = 'OIII1'
    else:
        line = 'OII'
    ra  = row[line+'_RA']
    dec = row[line+'_DEC']
    c = SkyCoord(ra, dec, unit='deg', frame='icrs')
    return c

hps['coord'] = hps.apply(build_astropy_coords, axis=1)

## Normalize LRS2 response from guider frames

In [85]:
#observation folder
obs_dir = '/Volumes/Briana_mac3/LRS2/response_normalization/20171223/lrs2/lrs20000024'

#guider directory 
gd1_dir = '/Volumes/Briana_mac3/LRS2/response_normalization/20171223/gc1'
gd2_dir = '/Volumes/Briana_mac3/LRS2/response_normalization/20171223/gc2'

#directory to store results
gc_dir = '/Volumes/Briana_mac3/LRS2/response_normalization/extract_gc'

In [86]:
#find observation date and time
obs_ims = glob.glob(os.path.join(obs_dir, 'exp*','lrs2','*066LL*fits'))
print obs_ims[0]
hdr = fits.getheader(obs_ims[0], 0) 
#print repr(hdr)
print hdr['UT']
print hdr['EXPTIME']

dt.datetime.strptime(hdr['DATE-OBS']+' '+hdr['UT'], '%Y-%m-%d %H:%M:%S.%f')

/Volumes/Briana_mac3/LRS2/response_normalization/20171223/lrs2/lrs20000024/exp01/lrs2/20171223T115219.1_066LL_sci.fits
11:52:19.171
1207.19892403


datetime.datetime(2017, 12, 23, 11, 52, 19, 171000)

In [87]:
t = tarfile.open(os.path.join(gd2_dir, 'images.tar'),'r')
tar_files = t.getmembers()
#members=[str(m).split('_')[0] for m in tar_files[0:5]]

#dt.datetime.strptime(date, '%Y%m%dT%H%M%S.%f')

#members=[m for m in t.getmembers() if str(m).split('\'')[1].split('_')[0]==date]
t.extractall(gc_dir, tar_files[0:2])

In [88]:
ext_gc = glob.glob(os.path.join(gc_dir, '*'))
print ext_gc[0]
hdr = fits.getheader(ext_gc[0], 0) 
#print repr(hdr)
print hdr['DATE-OBS']
print hdr['OBJECT']
print hdr['RA']
print hdr['DEC']

# c = SkyCoord(hdr['RA'], hdr['DEC'], unit='deg', frame='icrs')
# KRNmag = hdr['KRNMAG'] #magnitude
# KRNapr = hdr['KRNAPER'] #apeture size

# #flags
# sat     = hdr['OBSATUR']  #is the star saturated 
# obgood  = hdr['OBGOOD']   #do the parameters look good
# nostar  = hdr['FLNOSTAR'] #no star found                                  
# tostar  = hdr['FLAMBIGU'] #too many stars                                 
# bord    = hdr['FLBORDER'] #object at image border                         
# badreg  = hdr['FLBADIMG'] #object in bad image region                     
# badbkgd = hdr['FLBADSKY'] #unreliable background                          
# tobig   = hdr['FLTOOBIG'] #fit radius too big 


/Volumes/Briana_mac3/LRS2/response_normalization/extract_gc/20171223T062518.3_gc2_sci.fits
2017-12-23
TIC:000000000454116386
4.300284
5.917852


In [118]:
ims = glob.glob('/Volumes/Briana_mac3/LRS2/response_normalization/20171223/test/*.fits')
#ims = glob.glob('/Volumes/Briana_mac3/LRS2/response_normalization/extract_gc/*gc2*.fits')
cat = []
for i in ims:
    hdr = fits.getheader(i, 0) 
    obj = hdr['OBJECT']
    if obj:
        print i.split('/')[-1], obj
        cat.append(obj)
        
print len(ims)
print len(cat)

2555
1721


# Blueberry Table 

In [48]:
bb_header = pd.read_fwf('/Volumes/Briana_mac3/HPS/comparison_populations/blueberries/yang2017_blueberries_header.dat', names=['Bytes','Format','Unit','Label','Explanations'] )
bb_table = pd.read_fwf('/Volumes/Briana_mac3/HPS/comparison_populations/blueberries/yang2017_blueberries_table.dat', names=bb_header['Label'], na_values=[-99.0, -999.0])

bb_table = bb_table[~np.isnan(bb_table['[OII]3727']) & ~np.isnan(bb_table['[OIII]5007']) & ~np.isnan(bb_table['Hbeta'])]

print bb_table.columns

print bb_table.loc[:,['ObjID','[OII]3727','[OIII]5007', 'Hbeta','e_[OII]3727','e_[OIII]5007', 'e_Hbeta']]
bb_table.to_csv('/Volumes/Briana_mac3/HPS/comparison_populations/blueberries/yang2017_blueberries.csv')

Index([u'ObjID', u'RAdeg', u'DEdeg', u'zspec', u'Tel', u'E(B-V)-MW', u'u-SDSS',
       u'e_u-SDSS', u'g-SDSS', u'e_g-SDSS', u'r-SDSS', u'e_r-SDSS', u'i-SDSS',
       u'e_i-SDSS', u'z-SDSS', u'e_z-SDSS', u'g-PS1', u'e_g-PS1', u'r-PS1',
       u'e_r-PS1', u'i-PS1', u'e_i-PS1', u'z-PS1', u'e_z-PS1', u'y-PS1',
       u'e_y-PS1', u'log(M/Msun)', u'12+log(O/H)', u'e_12+log(O/H)', u'SFR',
       u'[OII]3727', u'e_[OII]3727', u'[NeIII]3869', u'e_[NeIII]3869',
       u'Hdelta', u'e_Hdelta', u'Hgamma', u'e_Hgamma', u'[OIII]4363',
       u'e_[OIII]4363', u'Hbeta', u'e_Hbeta', u'[OIII]4959', u'e_[OIII]4959',
       u'[OIII]5007', u'e_[OIII]5007', u'Halpha', u'e_Halpha',
       u'EW([OII]3727)', u'EW(Hbeta)', u'e_EW(Hbeta)', u'EW([OIII]5007)',
       u'e_EW([OIII]5007)', u'EW(Halpha)', u'RAdeg-neighbor',
       u'DECdeg-neighbor', u'Dist-neighbor'],
      dtype='object')
    ObjID  [OII]3727  [OIII]5007    Hbeta  e_[OII]3727  e_[OIII]5007  e_Hbeta
0       2      6.456     137.485   22.784        0.

# Blue Compact Dwarf Table

In [50]:
bcd_fluxes = pd.read_table('/Volumes/Briana_mac3/HPS/comparison_populations/blue_compact_dwarfs/Lian2016_bcd_table.dat', delim_whitespace=True)
bcd_fluxes['[NeIII]3869'] = np.nan
bcd_fluxes['[NeIII]3869_e'] = np.nan
bcd_props  = pd.read_table('/Volumes/Briana_mac3/HPS/comparison_populations/blue_compact_dwarfs/Lian2016_bcd_table_met.dat', delim_whitespace=True)
bcd_merge = pd.merge(bcd_fluxes, bcd_props, on='ID', how='outer')
print bcd_merge.columns
print bcd_merge.loc[:,['ID', '[OII]3727', '[OIII]5007','Hbeta', '[NeIII]3869','logM*']]

bcd_merge.to_csv('/Volumes/Briana_mac3/HPS/comparison_populations/blue_compact_dwarfs/Lian2016_bcd_table.csv')

Index([u'ID', u'RA', u'DEC', u'[OII]3727', u'[OII]3727_e', u'Hgamma',
       u'Hgamma_e', u'Hbeta', u'Hbeta_e', u'EW(Hbeta)', u'[OIII]4959',
       u'[OIII]4959_e', u'[OIII]5007', u'[OIII]5007_e', u'[NeIII]3869',
       u'[NeIII]3869_e', u'z', u'Av', u'logM*', u'12+log(O/H)',
       u'12+log(O/H)_e', u'log(SFR)', u'log(SFR)_e', u'D_n(4000)'],
      dtype='object')
    ID  [OII]3727  [OIII]5007  Hbeta  [NeIII]3869  logM*
0    1      142.3       298.9   65.8          NaN   8.29
1    2      155.8       206.7   54.6          NaN   8.38
2    3      285.1       187.8   51.9          NaN   8.79
3    4      273.0       248.7   75.6          NaN   8.73
4    5       56.4       223.1   37.7          NaN   7.84
5    6      121.3       287.9   52.0          NaN   8.35
6    7      113.3        84.5   25.3          NaN   8.52
7    8      200.4       302.0   56.6          NaN   8.48
8    9      227.7      1010.0  133.8          NaN   8.74
9   10       94.7        68.3   29.1          NaN   8.14
10  11

# Green Peas Table

In [51]:
gp_fluxes = pd.read_csv('/Volumes/Briana_mac3/HPS/comparison_populations/green_peas/Hawley2012_gp_fluxes.csv', na_values = '...')
gp_card = pd.read_table('/Volumes/Briana_mac3/HPS/comparison_populations/green_peas/green_pea_table.ascii', delim_whitespace=True)
gp_mass = gp_card.loc[:,['SDSS_ID', 'logM', 'z','metal']]

gp_merge = pd.merge(gp_fluxes, gp_mass, on='SDSS_ID', how='outer')
print len(gp_fluxes), len(gp_mass), len(gp_merge)

gp_merge = gp_merge[~np.isnan(gp_merge['[OII]3727']) & ~np.isnan(gp_merge['[OIII]5007']) & ~np.isnan(gp_merge['Hbeta']) & ~np.isnan(gp_merge['logM'])]
print len(gp_merge)
print gp_merge.columns
print gp_merge

gp_merge.to_csv('/Volumes/Briana_mac3/HPS/comparison_populations/green_peas/Hawley2012_Cardamone2009_gp_table.csv')

80 80 80
71
Index([u'SDSS_ID', u'[OII]3727', u'[NeIII]3868', u'Hdelta', u'Hgamma',
       u'[OIII]4363', u'HeI4471', u'HeII4686', u'Hbeta', u'[OIII]4959',
       u'[OIII]5007', u'logM', u'z', u'metal'],
      dtype='object')
               SDSS_ID  [OII]3727  [NeIII]3868 Hdelta  Hgamma  [OIII]4363  \
0   587725073921409255      243.0         33.1   26.9    49.7         NaN   
1   588848899919446344      145.0         46.9   26.6    48.3        5.44   
2   587725576962244831      193.0         48.0   33.4    54.9        7.84   
3   587731187273892048      272.0         43.2   26.8    50.2        2.74   
4   587731513693503653      246.0         37.5   20.8    55.6         NaN   
5   587724233716596882      234.0         32.4   21.4    44.3         NaN   
6   587727179006148758       86.8         40.2   22.6    43.1       11.40   
7   587724241767825591      195.0         42.2   25.1    46.6        4.90   
8   587724240158589061      225.0         38.5   23.7    45.1        2.85   
9   5

# Typical Low Mass Galaxies 

In [52]:
lmg_met  = pd.read_table('/Volumes/Briana_mac3/HPS/comparison_populations/typical_low_mass/berg2012_LMG_met.dat', delim_whitespace=True)
lmg_mass = pd.read_table('/Volumes/Briana_mac3/HPS/comparison_populations/typical_low_mass/berg2012_LMG_mass.dat', delim_whitespace=True)


print lmg_met
print lmg_mass
lmg_merge = pd.merge(lmg_met, lmg_mass, on='Galaxy', how='left')

print len(lmg_merge)
print lmg_merge.columns
print lmg_merge.head(5)

lmg_merge.to_csv('/Volumes/Briana_mac3/HPS/comparison_populations/typical_low_mass/berg2012_LMG_table.csv')

      Galaxy        R.A.      Decl.  F_[4.5]  F_[ks]      D    D_e  Ref  \
0        WLM  00:01:58.6  -15:27:12    62.90   117.0  0.970  0.020    1   
1      NGC55  00:14:53.6  -39:11:48  1390.00  2630.0  2.110  0.040    2   
2   UGC00668  01:04:49.1   02:07:31    90.10   232.0  0.750  0.020    1   
3    NGC1705  04:54:13.7  -53:21:41    19.30    44.4  5.110  0.170    3   
4    NGC2366  07:28:49.6   69:12:32     4.99   110.0  3.210  0.050    2   
5    UGC4305  08:19:09.0   70:43:28    64.60   216.0  3.380  0.050    2   
6    UGC4459  08:34:07.6   66:10:39     3.10    7.93  3.610  0.050    2   
7       LeoA  09:59:24.8   30:44:49    13.60    34.8  0.810  0.040    4   
8       SexB  10:00:00.0   05:19:56    36.10   136.0  1.390  0.040    2   
9       SexA  10:11:00.7  -04:41:37    24.90    62.2  1.380  0.050    2   
10   UGC5666  10:28:35.3   68:25:53   111.00   165.0  3.790  0.050    2   
11   NGC4214  12:15:39.0   36:19:35   224.00   488.0  3.030  0.050    2   
12   UGC8091  12:58:39.8 